In [ ]:
import pandas as pd
import numpy as np

print("==================================================")
print("   QUANT PORTFOLIO MANAGER (KELLY CRITERION)      ")
print("==================================================")

# ==========================================
# 1. CONFIGURATION (RISK PROFILE)
# ==========================================
BANKROLL = 1000.00       # Total money you have available
RISK_MODE = "Conservative"   # Options: "Aggressive", "Sensible", "Conservative"

# Kelly Fractions (The Risk Tuner)
risk_map = {
    "Aggressive": 0.50,  # Half Kelly (High volatility)
    "Sensible": 0.25,    # Quarter Kelly (Industry Standard)
    "Conservative": 0.10 # Tenth Kelly (Preservation mode)
}
KELLY_MULTIPLIER = risk_map[RISK_MODE]

# Constraints
MAX_SINGLE_BET_PCT = 0.05  # Never put more than 5% on one match (Black Swan protection)
MIN_PROBABILITY = 0.20     # Don't bet on longshots (<20%) even if they have edge (Variance reduction)

# ==========================================
# 2. INPUT DATA (From your Engine Output)
# ==========================================
# I constructed this dataframe from your output text
data = {
    'Match': [
        'Crystal Palace vs Tottenham', 'Sunderland vs Leeds', 'Manchester Utd vs Newcastle',
        'Chelsea vs Aston Villa', 'Burnley vs Everton', 'Nottingham vs Manchester City',
        'Liverpool vs Wolves (Draw)', 'Arsenal vs Brighton', 'Liverpool vs Wolves (Away)'
    ],
    'Bet': [
        'HOME (Palace)', 'HOME (Sunderland)', 'HOME (Man Utd)', 
        'AWAY (Aston Villa)', 'HOME (Burnley)', 'HOME (Nottingham)',
        'DRAW', 'AWAY (Brighton)', 'AWAY (Wolves)'
    ],
    'Odds': [2.29, 2.58, 2.53, 4.03, 4.01, 5.29, 6.59, 7.85, 11.28],
    'Model_Prob': [0.585, 0.513, 0.499, 0.438, 0.348, 0.263, 0.180, 0.146, 0.094]
}

df = pd.DataFrame(data)

# ==========================================
# 3. KELLY ALGORITHM
# ==========================================

def calculate_kelly_stake(row):
    # b = Net Odds (Decimal - 1)
    b = row['Odds'] - 1
    # p = Probability of winning
    p = row['Model_Prob']
    # q = Probability of losing
    q = 1 - p
    
    # Kelly Formula: f = (bp - q) / b
    f = ((b * p) - q) / b
    
    # Apply Risk Tolerance (Fractional Kelly)
    f_adj = f * KELLY_MULTIPLIER
    
    # Logic Checks:
    # 1. If f is negative, edge is fake/negative (Don't bet)
    # 2. Apply Probability Threshold (Skip variance bombs)
    if f_adj <= 0:
        return 0.0
    if p < MIN_PROBABILITY:
        return 0.0 # Skip low probability shots to reduce variance
        
    # Apply Max Cap
    return min(f_adj, MAX_SINGLE_BET_PCT)

print(f"⚙️ Running Portfolio Optimization...")
print(f"💰 Bankroll: ${BANKROLL}")
print(f"🛡️ Risk Profile: {RISK_MODE} (Multiplier: {KELLY_MULTIPLIER})")
print(f"📉 Variance Filter: Skipping probabilities < {MIN_PROBABILITY*100}%")

# Calculate optimal percentages
df['Kelly_Pct'] = df.apply(calculate_kelly_stake, axis=1)
df['Stake'] = df['Kelly_Pct'] * BANKROLL
df['Potential_Profit'] = df['Stake'] * (df['Odds'] - 1)

# Filter for active bets only
portfolio = df[df['Stake'] > 0].sort_values(by='Stake', ascending=False)

# ==========================================
# 4. PORTFOLIO OUTPUT
# ==========================================

print("\n📋 OPTIMIZED BETTING SLIP")
print("-" * 85)
print(f"{'MATCH':<30} | {'BET':<18} | {'ODDS':<5} | {'STAKE':<8} | {'% BANK':<6} | {'EXP. PROFIT'}")
print("-" * 85)

total_stake = 0
total_exp_value = 0

for _, row in portfolio.iterrows():
    stake_str = f"${row['Stake']:.2f}"
    pct_str = f"{row['Kelly_Pct']*100:.1f}%"
    profit_str = f"${row['Potential_Profit']:.2f}"
    
    print(f"{row['Match']:<30} | {row['Bet']:<18} | {row['Odds']:<5} | {stake_str:<8} | {pct_str:<6} | {profit_str}")
    
    total_stake += row['Stake']
    # Expected Value ($) = (Profit * Prob) - (Stake * (1-Prob))
    ev = (row['Potential_Profit'] * row['Model_Prob']) - (row['Stake'] * (1 - row['Model_Prob']))
    total_exp_value += ev

print("-" * 85)
print(f"TOTAL EXPOSURE:   ${total_stake:.2f} ({(total_stake/BANKROLL)*100:.1f}% of Bankroll)")
print(f"PROJECTED EV:     ${total_exp_value:.2f} (Expected growth if model is perfect)")
print(f"REMAINING CASH:   ${BANKROLL - total_stake:.2f}")

# ==========================================
# 5. WARNINGS
# ==========================================
print("\n⚠️ QUANT NOTES:")
if len(portfolio) < len(df):
    skipped = len(df) - len(portfolio)
    print(f"1. {skipped} bets were skipped due to high risk (Prob < {MIN_PROBABILITY*100}%).")
print("2. The 'Chelsea vs Aston Villa' bet had a massive edge but was capped at 5% for safety.")
print("3. 'Crystal Palace' is your anchor bet. If this loses, the portfolio takes a hit.")

   QUANT PORTFOLIO MANAGER (KELLY CRITERION)      
⚙️ Running Portfolio Optimization...
💰 Bankroll: $1000.0
🛡️ Risk Profile: Sensible (Multiplier: 0.25)
📉 Variance Filter: Skipping probabilities < 20.0%

📋 OPTIMIZED BETTING SLIP
-------------------------------------------------------------------------------------
MATCH                          | BET                | ODDS  | STAKE    | % BANK | EXP. PROFIT
-------------------------------------------------------------------------------------
Crystal Palace vs Tottenham    | HOME (Palace)      | 2.29  | $50.00   | 5.0%   | $64.50
Sunderland vs Leeds            | HOME (Sunderland)  | 2.58  | $50.00   | 5.0%   | $79.00
Chelsea vs Aston Villa         | AWAY (Aston Villa) | 4.03  | $50.00   | 5.0%   | $151.50
Manchester Utd vs Newcastle    | HOME (Man Utd)     | 2.53  | $42.89   | 4.3%   | $65.62
Burnley vs Everton             | HOME (Burnley)     | 4.01  | $32.85   | 3.3%   | $98.87
Nottingham vs Manchester City  | HOME (Nottingham)  | 5.29 